In [2]:
 pip install -U scikit-fuzzy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894079 sha256=69cb52e9b587b0c34120fa7f71d0efd249b8766b12a5083546f4152a90b4b5e5
  Stored in directory: /root/.cache/pip/wheels/4f/86/1b/dfd97134a2c8313e519bcebd95d3fedc7be7944db022094bc8
Successfully built scikit-fuzzy


In [ ]:
import pandas as pd
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import numpy as np

df = pd.read_csv("kz.csv")
df = df.head(1000)

category_code = ctrl.Antecedent(np.arange(len(df['category_code'].unique())), 'category_code')
price = ctrl.Antecedent(np.arange(df['price'].min(), df['price'].max() + 1, 1), 'price')
user_id = ctrl.Antecedent(np.arange(df['user_id'].min(), df['user_id'].max() + 1, 1), 'user_id')

unique_brands = df['brand'].unique()
brand = ctrl.Consequent(np.arange(len(unique_brands)), 'brand')

category_code['low'] = fuzz.trimf(category_code.universe, [0, 0, 3])
category_code['medium'] = fuzz.trimf(category_code.universe, [2, 5, 8])
category_code['high'] = fuzz.trimf(category_code.universe, [7, 10, 10])


price['low'] = fuzz.trimf(price.universe, [0, 100, 500])
price['medium'] = fuzz.trimf(price.universe, [100, 500, 1000])
price['high'] = fuzz.trimf(price.universe, [500, 1000, 1500])

user_id['low'] = fuzz.trimf(user_id.universe, [df['user_id'].min(), df['user_id'].min(), df['user_id'].min() + 1000])
user_id['medium'] = fuzz.trimf(user_id.universe, [df['user_id'].min() + 1000, df['user_id'].min() + 2000, df['user_id'].min() + 3000])
user_id['high'] = fuzz.trimf(user_id.universe, [df['user_id'].min() + 2000, df['user_id'].max(), df['user_id'].max()])


for idx, brand_name in enumerate(unique_brands):
    brand[brand_name] = fuzz.trimf(brand.universe, [idx, idx, idx])

rules = []

category_mapping = {
    'electronics.tablet': 'low',
    'electronics.audio.headphone': 'low',
    'karcher': 'low',
    'furniture.kitchen.table': 'medium',
    'electronics.smartphone': 'medium',
    'appliances.kitchen.refrigerators': 'high',
    'appliances.personal.scales': 'high',
    'electronics.video.tv': 'medium',
    'computers.components.cpu': 'medium',
}

for user in df['user_id'].unique():
    for category_code_name in df['category_code'].unique():
        data_subset = df[(df['user_id'] == user) & (df['category_code'] == category_code_name)]

        if not data_subset.empty:
            mode_result = data_subset['brand'].mode()
            if not mode_result.empty:
                brand_name = mode_result.iloc[0]

                category_term = category_mapping.get(category_code_name, 'high')

                rules.append(ctrl.Rule(category_code[category_term] & price['low'] & user_id['low'], brand[brand_name]))

brand_ctrl = ctrl.ControlSystem(rules)

brand_sim = ctrl.ControlSystemSimulation(brand_ctrl)